# Imports

In [292]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

# Data Vizs
import matplotlib.pyplot as plt
%matplotlib inline

# There is a bug in matplotlib. You cannot set the rc parameters in the same
# cell that you use the "%matplotlib inline" magic command
plt.style.use('ggplot')
plt.rc('font', size=18)

# Postgres info to connect
connection_args = {
    'host': '3.16.149.40',  # We are connecting to our _local_ version of psql
    'user':'ubuntu',
    'dbname': 'realestate',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

# Magic Python trick!
connection = pg.connect(**connection_args)
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Query to AWS 

In [50]:
query = "SELECT * FROM inventorymetrics;"

inventory_df = pd_sql.read_sql(query, connection)

In [71]:
OG_inventory_df = inventory_df.copy()

# Inventory DataFrame Cleaning

In [143]:
inventory_df = OG_inventory_df.copy()

In [144]:
# DF Update: update month to datetime

inventory_df['month'] = pd.to_datetime(inventory_df['month'])

In [145]:
# DF Update: update zipcode to int

inventory_df['zipcode'] = inventory_df['zipcode'].astype(int)

In [146]:
# DF Updates: remove footnote column

inventory_df = inventory_df.drop(columns = ['footnote'] )

In [147]:
inventory_df.columns

Index(['month', 'zipcode', 'zipname', 'medianlistingprice',
       'medianlistingprice_mm', 'medianlistingprice_yy', 'activelistingcount',
       'activelistingcount_mm', 'activelistingcount_yy', 'daysonmarket',
       'daysonmarket_mm', 'daysonmarket_yy', 'newlistingcount',
       'newlistingcount_mm', 'newlistingcount_yy', 'priceincreasecount',
       'priceincreasecount_mm', 'priceincreasecount_yy', 'pricedecreasecount',
       'pricedecreasecount_mm', 'pricedecreasecount_yy', 'pendinglistingcount',
       'pendinglistingcount_mm', 'pendinglistingcount_yy', 'avglistingprice',
       'avglistingprice_mm', 'avglistingprice_yy', 'totallistingcount',
       'totallistingcount_mm', 'totallistingcount_yy', 'pendingratio',
       'pendingratio_mm', 'pendingratio_yy'],
      dtype='object')

In [148]:
# DF Updates: separate zipname column into two columns (name, city)

# new data frame with split value columns 
new = inventory_df["zipname"].str.split(",", n = 2, expand = True) 
  
# making separate first name column from new data frame 
inventory_df["City"]= new[0] 
  
# making separate last name column from new data frame 
inventory_df["State"]= new[1] 

In [149]:
# DF Updates: rename zipname column name to city-state

inventory_df = inventory_df.rename(columns={"zipname": "city-state"})

In [150]:
inventory_df.columns

Index(['month', 'zipcode', 'city-state', 'medianlistingprice',
       'medianlistingprice_mm', 'medianlistingprice_yy', 'activelistingcount',
       'activelistingcount_mm', 'activelistingcount_yy', 'daysonmarket',
       'daysonmarket_mm', 'daysonmarket_yy', 'newlistingcount',
       'newlistingcount_mm', 'newlistingcount_yy', 'priceincreasecount',
       'priceincreasecount_mm', 'priceincreasecount_yy', 'pricedecreasecount',
       'pricedecreasecount_mm', 'pricedecreasecount_yy', 'pendinglistingcount',
       'pendinglistingcount_mm', 'pendinglistingcount_yy', 'avglistingprice',
       'avglistingprice_mm', 'avglistingprice_yy', 'totallistingcount',
       'totallistingcount_mm', 'totallistingcount_yy', 'pendingratio',
       'pendingratio_mm', 'pendingratio_yy', 'City', 'State'],
      dtype='object')

In [151]:
# DF Updates: Fill na values with 0
inventory_df = inventory_df.fillna(0)

In [152]:
# remove white space in State column
inventory_df['State'] = inventory_df['State'].str.strip()

In [153]:
# State column names
inventory_df['State'].unique()

array(['MA', 'RI', 'NH', 'VT', 'ME', 'CT', 'NJ', 'MS', 'NY', 'PA', 'VA',
       'DE', 'DC', 'MD', 'WV', 'NC', 'SC', 'AL', 'TN', 'GA', 'FL', 'KY',
       'OH', 'IN', 'MI', 'IA', 'WI', 'MN', 'SD', 'ND', 'MT', 'IL', 'MO',
       'KS', 'NE', 'LA', 'AR', 'OK', 'TX', 'CO', 'WY', 'ID', 'UT', 'AZ',
       'NM', 'NV', 'CA', 'HI', 'OR', 'WA', 'AK'], dtype=object)

In [154]:
# add a month column
inventory_df['month_no']= pd.to_datetime(inventory_df['month'], format='%Y:%M:%D').dt.month
inventory_df.head()

,month,zipcode,city-state,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,...,avglistingprice_yy,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,City,State,month_no
0,2019-04-01,1001,"Agawam, MA",213700.0,0.0739,-0.0904,43.5,-0.0543,-0.2627,49.50,...,-0.0602,44.5,-0.0532,-0.2583,0.0230,0.0012,0.0060,Agawam,MA,4
1,2019-04-01,1002,"Amherst, MA",499050.0,0.0398,0.1607,70.5,0.1190,-0.2079,43.00,...,0.0733,70.5,0.1190,-0.2079,0.0000,0.0000,0.0000,Amherst,MA,4
2,2019-04-01,1005,"Barre, MA",301275.0,-0.0024,0.0000,17.0,0.2143,0.1333,45.75,...,0.4213,17.0,0.2143,0.1333,0.0000,0.0000,0.0000,Barre,MA,4
3,2019-04-01,1007,"Belchertown, MA",369950.0,0.0291,0.0000,51.5,0.0300,-0.3041,47.50,...,0.0738,51.5,0.0300,-0.3041,0.0000,0.0000,0.0000,Belchertown,MA,4
4,2019-04-01,1008,"Blandford, MA",181950.0,-0.1915,-0.2159,6.5,-0.0714,0.0833,34.25,...,-0.2975,8.5,0.0625,-0.0556,0.3077,0.1648,-0.1923,Blandford,MA,4


In [156]:
# add a year column
inventory_df['year']= pd.to_datetime(inventory_df['month'], format='%Y:%M:%D').dt.year

In [157]:
# update month column
inventory_df = inventory_df.rename(columns={"month": "entry_date"})

In [158]:
inventory_df.head()

,entry_date,zipcode,city-state,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,...,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,City,State,month_no,year
0,2019-04-01,1001,"Agawam, MA",213700.0,0.0739,-0.0904,43.5,-0.0543,-0.2627,49.50,...,44.5,-0.0532,-0.2583,0.0230,0.0012,0.0060,Agawam,MA,4,2019
1,2019-04-01,1002,"Amherst, MA",499050.0,0.0398,0.1607,70.5,0.1190,-0.2079,43.00,...,70.5,0.1190,-0.2079,0.0000,0.0000,0.0000,Amherst,MA,4,2019
2,2019-04-01,1005,"Barre, MA",301275.0,-0.0024,0.0000,17.0,0.2143,0.1333,45.75,...,17.0,0.2143,0.1333,0.0000,0.0000,0.0000,Barre,MA,4,2019
3,2019-04-01,1007,"Belchertown, MA",369950.0,0.0291,0.0000,51.5,0.0300,-0.3041,47.50,...,51.5,0.0300,-0.3041,0.0000,0.0000,0.0000,Belchertown,MA,4,2019
4,2019-04-01,1008,"Blandford, MA",181950.0,-0.1915,-0.2159,6.5,-0.0714,0.0833,34.25,...,8.5,0.0625,-0.0556,0.3077,0.1648,-0.1923,Blandford,MA,4,2019


# Illinois DataFrame

In [201]:
# Illinois DataFrame

il_df = inventory_df[inventory_df['State'] == 'IL']
il_df.head()

,entry_date,zipcode,city-state,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,...,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,City,State,month_no,year
9887,2019-04-01,60002,"Antioch, IL",250000.00,-0.0198,0.0038,186.0,0.1481,-0.1878,44.75,...,190.0,0.1446,-0.2149,0.0215,-0.0032,-0.0353,Antioch,IL,4,2019
9888,2019-04-01,60004,"Arlington Heights, IL",349956.25,0.0000,-0.0669,407.0,0.1274,0.0970,38.00,...,428.5,0.1306,0.0959,0.0528,0.0030,-0.0011,Arlington Heights,IL,4,2019
9889,2019-04-01,60005,"Arlington Heights, IL",355725.00,0.0047,-0.0489,232.5,0.2109,0.0764,35.50,...,248.0,0.2039,0.1121,0.0667,-0.0063,0.0343,Arlington Heights,IL,4,2019
9890,2019-04-01,60007,"Elk Grove Village, IL",299950.00,0.0166,0.0909,199.5,0.2547,0.2627,37.25,...,205.0,0.2424,0.2349,0.0276,-0.0102,-0.0231,Elk Grove Village,IL,4,2019
9891,2019-04-01,60008,"Rolling Meadows, IL",249950.00,0.0869,0.0870,142.0,0.1545,0.0290,38.25,...,148.5,0.1423,0.0385,0.0458,-0.0111,0.0095,Rolling Meadows,IL,4,2019


In [202]:
inventory_df[inventory_df['zipcode'] == 60419]

,entry_date,zipcode,city-state,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,...,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,City,State,month_no,year
10025,2019-04-01,60419,"Dolton, IL",99750.00,-0.0020,0.2307,148.5,-0.0294,-0.0100,66.00,...,181.5,0.0028,-0.0082,0.2222,0.0392,0.0022,Dolton,IL,4,2019
25053,2019-03-01,60419,"Dolton, IL",99950.00,0.0571,0.2313,153.0,-0.0699,0.0200,72.00,...,181.0,-0.0398,-0.0398,0.1830,0.0371,-0.0737,Dolton,IL,3,2019
40082,2019-02-01,60419,"Dolton, IL",94550.00,-0.0076,0.1811,164.5,-0.0408,0.0186,87.75,...,188.5,-0.0456,-0.0284,0.1459,-0.0057,-0.0553,Dolton,IL,2,2019
55108,2019-01-01,60419,"Dolton, IL",95275.00,0.0024,0.2973,171.5,-0.0878,-0.0311,83.25,...,197.5,-0.0814,-0.0319,0.1516,0.0080,-0.0009,Dolton,IL,1,2019
70138,2018-12-01,60419,"Dolton, IL",95050.00,0.0276,0.2836,188.0,-0.0481,0.0503,75.00,...,215.0,-0.0138,0.0539,0.1436,0.0398,0.0040,Dolton,IL,12,2018
85166,2018-11-01,60419,"Dolton, IL",92500.00,0.0306,0.2248,197.5,0.0450,0.0942,65.75,...,218.0,0.0307,0.0307,0.1038,-0.0153,-0.0679,Dolton,IL,11,2018
100193,2018-10-01,60419,"Dolton, IL",89750.00,-0.0032,0.1959,189.0,0.1118,0.0500,64.50,...,211.5,0.0959,0.0024,0.1190,-0.0162,-0.0532,Dolton,IL,10,2018
115222,2018-09-01,60419,"Dolton, IL",90040.00,-0.0424,0.2230,170.0,0.0592,-0.0449,65.00,...,193.0,0.0321,-0.0981,0.1353,-0.0298,-0.0670,Dolton,IL,9,2018
130254,2018-08-01,60419,"Dolton, IL",94025.00,0.0453,0.2528,160.5,-0.0273,-0.1301,63.50,...,187.0,-0.0209,-0.1481,0.1651,0.0075,-0.0246,Dolton,IL,8,2018
145287,2018-07-01,60419,"Dolton, IL",89950.00,0.0576,0.1379,165.0,0.0248,-0.1176,66.50,...,191.0,-0.0026,-0.1198,0.1576,-0.0319,-0.0029,Dolton,IL,7,2018


In [161]:
# Return a month of Illinois data (03/2019 to 04/2019)

il_df[(il_df['entry_date'] >= '2019-03-01') & (il_df['entry_date'] <= '2019-04-01')]

,entry_date,zipcode,city-state,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,...,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,City,State,month_no,year
9887,2019-04-01,60002,"Antioch, IL",250000.00,-0.0198,0.0038,186.0,0.1481,-0.1878,44.75,...,190.0,0.1446,-0.2149,0.0215,-0.0032,-0.0353,Antioch,IL,4,2019
9888,2019-04-01,60004,"Arlington Heights, IL",349956.25,0.0000,-0.0669,407.0,0.1274,0.0970,38.00,...,428.5,0.1306,0.0959,0.0528,0.0030,-0.0011,Arlington Heights,IL,4,2019
9889,2019-04-01,60005,"Arlington Heights, IL",355725.00,0.0047,-0.0489,232.5,0.2109,0.0764,35.50,...,248.0,0.2039,0.1121,0.0667,-0.0063,0.0343,Arlington Heights,IL,4,2019
9890,2019-04-01,60007,"Elk Grove Village, IL",299950.00,0.0166,0.0909,199.5,0.2547,0.2627,37.25,...,205.0,0.2424,0.2349,0.0276,-0.0102,-0.0231,Elk Grove Village,IL,4,2019
9891,2019-04-01,60008,"Rolling Meadows, IL",249950.00,0.0869,0.0870,142.0,0.1545,0.0290,38.25,...,148.5,0.1423,0.0385,0.0458,-0.0111,0.0095,Rolling Meadows,IL,4,2019
9892,2019-04-01,60010,"Barrington, IL",618050.00,-0.0112,-0.0031,771.0,0.0844,0.0447,70.00,...,790.5,0.0770,0.0415,0.0253,-0.0071,-0.0032,Barrington,IL,4,2019
9893,2019-04-01,60012,"Crystal Lake, IL",335050.00,-0.0133,-0.0640,124.5,0.1528,0.2327,58.25,...,139.5,0.1625,0.3160,0.1205,0.0094,0.0710,Crystal Lake,IL,4,2019
9894,2019-04-01,60013,"Cary, IL",284900.00,0.0540,0.0177,201.0,0.0579,0.1292,40.50,...,210.0,0.0769,0.1351,0.0448,0.0185,0.0055,Cary,IL,4,2019
9895,2019-04-01,60014,"Crystal Lake, IL",274925.00,0.0665,-0.0503,325.0,0.1691,0.0220,37.00,...,339.5,0.1707,0.0226,0.0446,0.0014,0.0006,Crystal Lake,IL,4,2019
9896,2019-04-01,60015,"Deerfield, IL",577250.00,-0.0324,-0.0932,330.5,0.0872,0.0459,52.75,...,351.5,0.1019,0.0556,0.0635,0.0142,0.0097,Deerfield,IL,4,2019


In [203]:
# Return 3 years of Illinois data (04/2016 to 04/2019)

il_df2 = il_df[(il_df['entry_date'] >= '2016-04-01') & (il_df['entry_date'] <= '2019-04-01')]

In [204]:
# create a new target column:  if the DOM_mm is above the median set equal to 1, otherwise set to 0

dom = inventory_df['daysonmarket_mm']
dom_il = il_df['daysonmarket_mm']

In [205]:
il_mean = dom_il.mean()

In [206]:
il_median = dom_il.median()
il_median

0.0321

In [228]:
# y_median - threshold is the median value of the DOM m/m
il_df.loc[il_df['daysonmarket_mm'] < il_median, 'y_median'] = 1 # property spending less time on market means higher demand
il_df.loc[il_df['daysonmarket_mm'] >= il_median, 'y_median'] = 0 # property spending more time on market means lower demand

In [229]:
# y_mean - threshold is the mean value of the DOM m/m
il_df.loc[il_df['daysonmarket_mm'] < il_mean, 'y_mean'] = 1
il_df.loc[il_df['daysonmarket_mm'] >= il_mean, 'y_mean'] = 0

In [278]:
il_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37011 entries, 9887 to 1045306
Data columns (total 39 columns):
entry_date                37011 non-null datetime64[ns]
zipcode                   37011 non-null int64
city-state                37011 non-null object
medianlistingprice        37011 non-null float64
medianlistingprice_mm     37011 non-null float64
medianlistingprice_yy     37011 non-null float64
activelistingcount        37011 non-null float64
activelistingcount_mm     37011 non-null float64
activelistingcount_yy     37011 non-null float64
daysonmarket              37011 non-null float64
daysonmarket_mm           37011 non-null float64
daysonmarket_yy           37011 non-null float64
newlistingcount           37011 non-null float64
newlistingcount_mm        37011 non-null float64
newlistingcount_yy        37011 non-null float64
priceincreasecount        37011 non-null float64
priceincreasecount_mm     37011 non-null float64
priceincreasecount_yy     37011 non-null float64


In [231]:
# convert target to integers

il_df['y_median'] = il_df['y_median'].astype(int)
il_df['y_mean'] = il_df['y_mean'].astype(int)

/Users/ronald-daley/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ronald-daley/anaconda3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [232]:
il_df['y_median'].value_counts()

0    18508
1    18503
Name: y_median, dtype: int64

In [233]:
il_df['y_mean'].value_counts()

0    20517
1    16494
Name: y_mean, dtype: int64

# Data Exploration

In [ ]:
# Groupby y_median - averages

In [263]:
y_median_gb = il_df.groupby('y_median').mean()

In [235]:
y_mean_gb = il_df.groupby('y_mean').mean()

In [236]:
y_median_gb.columns

Index(['zipcode', 'medianlistingprice', 'medianlistingprice_mm',
       'medianlistingprice_yy', 'activelistingcount', 'activelistingcount_mm',
       'activelistingcount_yy', 'daysonmarket', 'daysonmarket_mm',
       'daysonmarket_yy', 'newlistingcount', 'newlistingcount_mm',
       'newlistingcount_yy', 'priceincreasecount', 'priceincreasecount_mm',
       'priceincreasecount_yy', 'pricedecreasecount', 'pricedecreasecount_mm',
       'pricedecreasecount_yy', 'pendinglistingcount',
       'pendinglistingcount_mm', 'pendinglistingcount_yy', 'avglistingprice',
       'avglistingprice_mm', 'avglistingprice_yy', 'totallistingcount',
       'totallistingcount_mm', 'totallistingcount_yy', 'pendingratio',
       'pendingratio_mm', 'pendingratio_yy', 'month_no', 'year', 'y_mean'],
      dtype='object')

In [264]:
# remove the extra columns
y_median_gb = y_median_gb.drop(columns = ['zipcode', 'month_no','year','y_mean'] )

In [265]:
# remove the DOM columns 
y_median_gb = y_median_gb.drop(columns = ['daysonmarket_mm', 'daysonmarket_yy','daysonmarket'] )

In [246]:
y_median_gb

,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,newlistingcount,newlistingcount_mm,newlistingcount_yy,priceincreasecount,...,pendinglistingcount_yy,avglistingprice,avglistingprice_mm,avglistingprice_yy,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy
y_median,,,,,,,,,,,,,,,,,,,,,
0,223590.962516,0.001248,0.035637,126.092095,-0.030437,-0.033643,29.984655,0.010060,0.009684,0.500648,...,0.019472,269326.671818,0.001818,0.046734,139.839043,-0.032384,0.000497,0.118416,-0.001338,0.021597
1,213290.906150,0.009565,0.037377,128.887018,0.033649,-0.044643,39.758742,0.182248,0.134297,0.639464,...,0.003870,255142.187429,0.006723,0.048006,144.226315,0.037018,-0.018375,0.129870,0.005558,0.019762


0 = DOM is equal to or above the median DOM for the market (properties in these markets have a  lower demand)

1 = DOM is below the median DOM for the market (properties in these markets have a higher demand)

In [249]:
# check for outliers
il_df.describe()

,zipcode,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,daysonmarket_mm,daysonmarket_yy,...,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,month_no,year,y_median,y_mean
count,37011.000000,3.701100e+04,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,...,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000,37011.000000
mean,60995.289644,2.184416e+05,0.005406,0.036507,127.489368,0.001601,-0.039143,80.728175,0.012861,-0.056816,...,142.032382,0.002312,-0.008938,0.124142,0.002110,0.020680,6.528573,2015.915295,0.499932,0.445651
std,846.631423,1.721676e+05,0.064346,0.107752,101.968085,0.115042,0.172655,33.767731,0.179506,0.188196,...,113.100345,0.104095,0.220309,0.183140,0.085221,0.243867,3.495705,1.729767,0.500007,0.497044
min,60002.000000,1.780000e+04,-0.899100,-0.555700,0.000000,-0.550000,-0.645200,14.000000,-0.885900,-0.822800,...,0.000000,-0.552600,-1.000000,0.000000,-1.497300,-2.091400,1.000000,2013.000000,0.000000,0.000000
25%,60410.000000,1.100500e+05,-0.020800,-0.015800,47.500000,-0.066300,-0.148900,57.000000,-0.078900,-0.175600,...,52.500000,-0.062900,-0.144100,0.000000,-0.011600,-0.022900,3.000000,2014.000000,0.000000,0.000000
50%,60634.000000,1.790500e+05,0.000000,0.018000,101.000000,-0.007200,-0.020700,74.000000,0.032100,-0.046100,...,112.500000,-0.006300,-0.031300,0.061300,0.000000,0.000000,7.000000,2016.000000,0.000000,0.000000
75%,61603.000000,2.667775e+05,0.026900,0.096700,180.000000,0.059150,0.035950,96.500000,0.122500,0.028300,...,202.000000,0.058800,0.096800,0.157350,0.014100,0.041100,10.000000,2017.000000,1.000000,1.000000
max,62995.000000,2.523050e+06,1.920800,1.328900,883.000000,1.294700,0.931800,380.000000,1.505100,1.035500,...,963.500000,1.615400,4.153800,2.333300,1.533300,2.333300,12.000000,2019.000000,1.000000,1.000000


In [251]:
# because all of the other features are continuous, I cannot do any other groupby, unless I do further feature engineering

# Data Visualization

In [253]:
# show plots in the notebook
%matplotlib inline

In [288]:
# Pairplot

In [289]:
sns.pairplot(il_df)

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x118d31158> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x118f911e0> (for post_execute):


KeyboardInterrupt: 

In [254]:
# bar plot with y-median and zipcode in the x-axis and values in y-axis

In [257]:
il_df.head()

,entry_date,zipcode,city-state,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,...,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,City,State,month_no,year,y_median,y_mean
9887,2019-04-01,60002,"Antioch, IL",250000.00,-0.0198,0.0038,186.0,0.1481,-0.1878,44.75,...,-0.2149,0.0215,-0.0032,-0.0353,Antioch,IL,4,2019,1,0
9888,2019-04-01,60004,"Arlington Heights, IL",349956.25,0.0000,-0.0669,407.0,0.1274,0.0970,38.00,...,0.0959,0.0528,0.0030,-0.0011,Arlington Heights,IL,4,2019,1,1
9889,2019-04-01,60005,"Arlington Heights, IL",355725.00,0.0047,-0.0489,232.5,0.2109,0.0764,35.50,...,0.1121,0.0667,-0.0063,0.0343,Arlington Heights,IL,4,2019,1,1
9890,2019-04-01,60007,"Elk Grove Village, IL",299950.00,0.0166,0.0909,199.5,0.2547,0.2627,37.25,...,0.2349,0.0276,-0.0102,-0.0231,Elk Grove Village,IL,4,2019,0,0
9891,2019-04-01,60008,"Rolling Meadows, IL",249950.00,0.0869,0.0870,142.0,0.1545,0.0290,38.25,...,0.0385,0.0458,-0.0111,0.0095,Rolling Meadows,IL,4,2019,1,1


In [272]:
# Groupby zipcode and year to find mean value per column
zip_year_gb_mean = il_df.groupby(['zipcode', 'year']).mean()
zip_year_gb_mean.head(13)

medianlistingprice  medianlistingprice_mm  \
zipcode year                                              
60002   2013       191858.333333               0.000183   
        2014       207231.250000               0.013567   
        2015       220687.458333              -0.007708   
        2016       221297.708333               0.004492   
        2017       237798.333333               0.007975   
        2018       243770.166667               0.003650   
        2019       252492.500000               0.010725   
60004   2013       320683.333333               0.002867   
        2014       348356.208333               0.009933   
        2015       356200.000000              -0.004267   
        2016       367745.812500               0.004825   
        2017       378502.083333               0.002667   
        2018       366471.875000              -0.005592   

              medianlistingprice_yy  activelistingcount  \
zipcode year                                              
60002   2013               0.009667          290.333333   
        2014               0.052100          250.708333   
        2015               0.076833          240.291667   
        2016               0.008950          215.166667   
        2017               0.074442          200.041667   
        2018               0.019950          210.041667   
        2019               0.047650          153.625000   
60004   2013               0.000000          273.500000   
        2014               0.027967          256.458333   
        2015               0.023642          247.541667   
        2016               0.031058          324.541667   
        2017               0.029775          328.125000   
        2018              -0.031458          343.083333   

              activelistingcount_mm  activelistingcount_yy  daysonmarket  \
zipcode year                                                               
60002   2013              -0.026900               0.179900     96.375000   
        2014              -0.020417              -0.035042     90.687500   
        2015               0.025217               0.029208     81.500000   
        2016              -0.015292              -0.080925     75.437500   
        2017               0.008133              -0.069333     64.187500   
        2018              -0.003825               0.008425     62.062500   
        2019               0.049025              -0.212725     65.125000   
60004   2013              -0.092550              -0.185750     66.958333   
        2014               0.006500              -0.087017     56.687500   
        2015               0.037542               0.002850     51.770833   
        2016               0.001117              -0.005333     52.854167   
        2017               0.013208               0.015058     48.791667   
        2018               0.016608               0.045600     45.708333   

              daysonmarket_mm  daysonmarket_yy  newlistingcount  ...  \
zipcode year                                                     ...   
60002   2013         0.088867        -0.092167        47.666667  ...   
        2014        -0.008475        -0.113750        56.166667  ...   
        2015         0.019475        -0.014258        53.333333  ...   
        2016         0.001758        -0.073692        49.166667  ...   
        2017        -0.007350        -0.144775        55.666667  ...   
        2018        -0.000150         0.007433        60.000000  ...   
        2019        -0.052100        -0.017175        52.500000  ...   
60004   2013         0.083600        -0.096950        65.666667  ...   
        2014         0.002758        -0.159525        89.000000  ...   
        2015         0.010250        -0.039133        97.333333  ...   
        2016         0.011833        -0.004325       107.333333  ...   
        2017         0.003742        -0.075283       119.500000  ...   
        2018         0.022683        -0.061583       135.333333  ...   

              avglis

In [277]:
# Groupby zip and year to find median value per column

zip_year_gb_median = il_df.groupby(['zipcode', 'year']).median()
zip_year_gb_median.head(7)

medianlistingprice  medianlistingprice_mm  \
zipcode year                                              
60002   2013            192287.5               -0.00120   
        2014            209500.0                0.00670   
        2015            222350.0                0.00620   
        2016            223250.0                0.00490   
        2017            240747.5                0.01265   
        2018            244912.5                0.00185   
        2019            252460.0                0.00150   

              medianlistingprice_yy  activelistingcount  \
zipcode year                                              
60002   2013                0.00440              300.00   
        2014                0.05140              240.25   
        2015                0.10190              245.50   
        2016                0.00600              219.25   
        2017                0.07090              199.25   
        2018                0.01005              217.00   
        2019                0.04480              149.75   

              activelistingcount_mm  activelistingcount_yy  daysonmarket  \
zipcode year                                                               
60002   2013               -0.03050                0.19115        93.125   
        2014               -0.01475                0.03245        83.500   
        2015                0.02145                0.00000        83.500   
        2016               -0.01050               -0.08465        69.500   
        2017                0.01175               -0.05395        57.875   
        2018               -0.01310                0.00525        59.750   
        2019                0.04315               -0.22235        63.250   

              daysonmarket_mm  daysonmarket_yy  newlistingcount  ...  \
zipcode year                                                     ...   
60002   2013          0.13710         -0.04520             49.0  ...   
        2014          0.02685         -0.11405             55.0  ...   
        2015          0.04615          0.03615             51.0  ...   
        2016          0.04610         -0.11425             50.0  ...   
        2017          0.04375         -0.14925             60.0  ...   
        2018          0.04240          0.02870             61.0  ...   
        2019          0.05895         -0.05595             57.0  ...   

              avglistingprice_yy  totallistingcount  totallistingcount_mm  \
zipcode year                                                                
60002   2013            -0.00415              300.0              -0.03050   
        2014             0.05265              278.0              -0.01475   
        2015             0.09430              287.5              -0.02375   
        2016             0.00390              229.0              -0.01675   
        2017             0.08880              214.0              -0.00200   
        2018             0.04845              225.0              -0.01560   
        2019             0.03100              154.5               0.03385   

              totallistingcount_yy  pendingratio  pendingratio_mm  \
zipcode year                                                        
60002   2013               0.19115       0.00000           0.0000   
        2014               0.03245       0.00000           0.0000   
        2015               0.05225       0.26735           0.0021   
        2016              -0.20325       0.05955           0.0002   
        2017              -0.03270       0.06405           0.0004   
        2018               0.02595       0.04420          -0.0017   
        2019              -0.24815       0.02400          -0.0009   

              pendingratio_yy  month_no  y_median  y_mean  
zipcode year                                               
60002   2013          0.00000       9.5       0.0     0.0  
        2014          0.00000       6.5       1.0     0.0  
        2015          0.26735       6.5       0.0     0.0  


In [274]:
# Groupby year to find median value per column
year_gb_median = il_df.groupby([ 'year']).median()
year_gb_median

,zipcode,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,daysonmarket_mm,daysonmarket_yy,...,avglistingprice_yy,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,month_no,y_median,y_mean
year,,,,,,,,,,,,,,,,,,,,,
2013,60634.0,149950.0,0.00000,0.00760,121.0,-0.0319,-0.08330,86.50,0.05140,-0.11220,...,0.04095,121.00,-0.03190,-0.08770,0.00000,0.0000,0.0000,9.0,0.0,0.0
2014,60635.0,159950.0,0.00040,0.01660,111.0,-0.0029,-0.01610,83.50,0.02820,-0.05425,...,0.06535,121.00,0.00405,0.01350,0.00000,0.0000,0.0000,6.5,1.0,0.0
2015,60634.0,172500.0,0.00000,0.01720,97.5,0.0000,0.00000,76.25,0.03740,-0.04210,...,0.04370,123.00,-0.00790,0.03720,0.13860,0.0000,0.0947,7.0,0.0,0.0
2016,60634.0,175500.0,0.00030,0.00310,106.0,-0.0100,0.00000,74.00,0.03175,0.00000,...,0.02575,117.00,-0.01025,-0.04965,0.07870,0.0000,-0.0230,6.5,1.0,0.0
2017,60634.0,189950.0,0.00020,0.03975,97.5,-0.0075,-0.08155,66.50,0.03370,-0.08080,...,0.04495,108.00,-0.00860,-0.08810,0.07890,0.0000,-0.0015,6.5,0.0,0.0
2018,60634.0,199950.0,0.00000,0.01995,92.5,0.0000,-0.03570,62.50,0.03330,-0.04730,...,0.02660,102.00,-0.00155,-0.04550,0.07605,0.0000,-0.0023,6.5,0.0,0.0
2019,60634.0,199950.0,0.00385,0.00000,82.0,0.0185,0.00000,70.50,-0.03775,0.00000,...,0.01360,92.75,0.03030,0.00810,0.07745,0.0036,0.0000,2.5,1.0,1.0


In [275]:
# Groupby year to find mean value per column
year_gb_mean = il_df.groupby([ 'year']).mean()
year_gb_mean

,zipcode,medianlistingprice,medianlistingprice_mm,medianlistingprice_yy,activelistingcount,activelistingcount_mm,activelistingcount_yy,daysonmarket,daysonmarket_mm,daysonmarket_yy,...,avglistingprice_yy,totallistingcount,totallistingcount_mm,totallistingcount_yy,pendingratio,pendingratio_mm,pendingratio_yy,month_no,y_median,y_mean
year,,,,,,,,,,,,,,,,,,,,,
2013,60995.678864,193297.169953,0.000367,0.034057,145.996688,-0.028462,-0.075177,90.632965,0.047128,-0.108760,...,0.043022,145.996688,-0.028462,-0.054833,0.000000,0.000000,0.000000,9.499369,0.432808,0.366877
2014,60996.007576,203897.381944,0.008428,0.044253,136.161379,-0.007076,-0.033382,90.314867,0.010434,-0.058340,...,0.069230,147.273990,0.009716,0.035038,0.087363,0.022937,0.087363,6.500000,0.511837,0.451231
2015,60995.268558,213991.946887,0.002291,0.037031,123.003783,0.017435,-0.033511,83.696533,0.015358,-0.056167,...,0.050254,154.354689,0.001773,0.055153,0.262717,-0.015354,0.175268,6.502128,0.484791,0.425374
2016,60995.022684,217749.079080,0.005833,0.025467,133.527804,-0.003373,-0.009083,80.913398,0.012279,-0.025756,...,0.032913,147.161468,-0.003025,-0.039067,0.105541,0.000693,-0.157125,6.500000,0.500473,0.445810
2017,60995.022684,228752.775638,0.006769,0.049343,123.154773,-0.002116,-0.077560,73.879608,0.015087,-0.078353,...,0.053856,135.883349,-0.002465,-0.063573,0.110776,-0.000429,0.005235,6.500000,0.496062,0.446597
2018,60995.022684,234809.788595,0.003565,0.034337,118.985744,0.007227,-0.037454,70.636578,0.017591,-0.052321,...,0.037027,131.205892,0.007210,-0.025150,0.115955,0.000402,0.005178,6.500000,0.496377,0.448330
2019,60995.022684,235043.532727,0.013404,0.016535,107.646267,0.034338,0.000713,78.547968,-0.057810,-0.018435,...,0.034717,118.988185,0.043514,0.038893,0.125188,0.012264,0.014698,2.500000,0.630907,0.596408


# Export Data

In [ ]:
df.to_csv(file_name, sep='\t')

In [296]:
# Export Illinois Data

il_df.to_excel('il_data3.xlsx', index=False, header = True)